In [0]:
import logging  # Use for logging errors or information, e.g., logging.error("message")
import requests  # Use for making HTTP requests to the weather API, e.g., requests.get(url)
import json  # Use for handling JSON data, e.g., json.dumps(data)
from azure.eventhub import *
 # Use for sending data to Azure Event Hub, e.g., EventHubProducerClient.from_connection_string(...)

EVENT_HUB_CONNECTION_STRING = dbutils.secrets.get(
    scope="key-vault-scope", key="event-hub-key"
)  # Use dbutils.secrets.get to securely retrieve secrets from Databricks

EVENT_HUB_NAME = "weather-event-hub"  # Use as the Event Hub name for Azure Event Hub

# Initialize the Event Hub producer client using the provided connection string and Event Hub name.
producer = EventHubProducerClient.from_connection_string(conn_str=EVENT_HUB_CONNECTION_STRING,eventhub_name=EVENT_HUB_NAME)  # Use EventHubProducerClient.from_connection_string to create a producer client


# Function to send a single event (as a dictionary) to Azure Event Hub.
def send_event(event):  # Use to send a dictionary event to Azure Event Hub
    # Create a batch to hold the event data. Batching is recommended for efficiency.
    event_data_batch = producer.create_batch()  # Use producer.create_batch() to create a batch
    # Serialize the event dictionary to a JSON string and add it to the batch.
    event_data_batch.add(EventData(json.dumps(event)))  # Use EventData(json.dumps(event)) to serialize and add to batch
    # Send the batch of events to the Event Hub.
    producer.send_batch(event_data_batch)  # Use producer.send_batch to send the batch
    print(f"sent the event to EVENT HUB: {event}")  # Use print to log the sent event


# Function to handle the API response
def handle_response(response):  # Handles HTTP response from API, use to process requests.get response
    if response.status_code == 200:  # Checks if response is successful, use to verify HTTP 200 OK
        return response.json()  # Returns JSON data if successful, use response.json() to parse JSON
    else:
        return f"Error: {response.status_code}, {response.text}"  # Returns error message if not successful, use for error handling


# Function to get current weather and air quality data
def get_current_weather(base_url, api_key, location):  # Fetches current weather data, use to call current weather endpoint
    current_weather_url = (
        f"{base_url}/current.json"  # Constructs URL for current weather endpoint, use f-string for endpoint
    )
    params = {
        "key": api_key,
        "q": location,
        "aqi": "yes",
    }  # Sets parameters for API call, use as params in requests.get
    response = requests.get(
        current_weather_url, params=params
    )  # Makes GET request to API, use requests.get(url, params=params)
    return handle_response(response)  # Handles and returns API response, use to process response


# Function to get Forecast Data
def get_forecast_weather(
    base_url, api_key, location, days
):  # Fetches weather forecast data, use to call forecast endpoint
    forecast_url = f"{base_url}/forecast.json"  # Constructs URL for forecast endpoint, use f-string for endpoint
    params = {
        "key": api_key,  # API key parameter, use as part of params
        "q": location,  # Location parameter, use as part of params
        "days": days,  # Number of days for forecast, use as part of params
    }
    response = requests.get(forecast_url, params=params)  # Makes GET request to API, use requests.get(url, params=params)
    return handle_response(response)  # Handles and returns API response, use to process response


# Function to get Alerts
def get_alerts(base_url, api_key, location):  # Fetches weather alerts, use to call alerts endpoint
    alerts_url = f"{base_url}/alerts.json"  # Constructs URL for alerts endpoint, use f-string for endpoint
    params = {
        "key": api_key,
        "q": location,
        "alerts": "yes",
    }  # Sets parameters for API call, use as params in requests.get
    response = requests.get(alerts_url, params=params)  # Makes GET request to API, use requests.get(url, params=params)
    return handle_response(response)  # Handles and returns API response, use to process response


# Flatten and merge the data
def flatten_data(
    current_weather, forecast_weather, alerts
):  # Flattens and merges API data, use to combine and flatten API responses
    location_data = current_weather.get("location", {})  # Extracts location data, use dict.get to access nested data
    current = current_weather.get("current", {})  # Extracts current weather data, use dict.get to access nested data
    condition = current.get("condition", {})  # Extracts weather condition, use dict.get to access nested data
    air_quality = current.get("air_quality", {})  # Extracts air quality data, use dict.get to access nested data
    forecast = forecast_weather.get("forecast", {}).get(
        "forecastday", []
    )  # Extracts forecast data, use nested dict.get to access list
    alert_list = alerts.get("alerts", {}).get("alert", [])  # Extracts alerts list, use nested dict.get to access list

    flattened_data = {
        "name": location_data.get("name"),  # City name, use dict.get to access
        "region": location_data.get("region"),  # Region name, use dict.get to access
        "country": location_data.get("country"),  # Country name, use dict.get to access
        "lat": location_data.get("lat"),  # Latitude, use dict.get to access
        "lon": location_data.get("lon"),
        "temp_c": current.get("temp_c"),  # Current temperature in Celsius
        "is_day": current.get("is_day"),  # Day or night indicator
        "condition_text": condition.get("text"),  # Weather condition text
        "condition_icon": condition.get("icon"),  # Weather condition icon URL
        "wind_kph": current.get("wind_kph"),  # Wind speed in kph
        "wind_degree": current.get("wind_degree"),  # Wind direction in degrees
        "wind_dir": current.get("wind_dir"),  # Wind direction as text
        "pressure_in": current.get("pressure_in"),  # Pressure in inches
        "precip_in": current.get("precip_in"),  # Precipitation in inches
        "humidity": current.get("humidity"),  # Humidity percentage
        "cloud": current.get("cloud"),  # Cloud cover percentage
        "feelslike_c": current.get("feelslike_c"),  # Feels like temperature in Celsius
        "uv": current.get("uv"),  # UV index
        "air_quality": {  # Air quality details
            "co": air_quality.get("co"),  # Carbon monoxide
            "no2": air_quality.get("no2"),  # Nitrogen dioxide
            "o3": air_quality.get("o3"),  # Ozone
            "so2": air_quality.get("so2"),  # Sulfur dioxide
            "pm2_5": air_quality.get("pm2_5"),  # PM2.5
            "pm10": air_quality.get("pm10"),  # PM10
            "us-epa-index": air_quality.get("us-epa-index"),  # US EPA index
            "gb-defra-index": air_quality.get("gb-defra-index"),  # UK DEFRA index
        },
        "alerts": [
            {
                "headline": alert.get("headline"),  # Alert headline
                "severity": alert.get("severity"),  # Alert severity
                "description": alert.get("desc"),  # Alert description
                "instruction": alert.get("instruction"),  # Alert instructions
            }
            for alert in alert_list  # Loops through all alerts
        ],
        "forecast": [
            {
                "date": day.get("date"),  # Forecast date
                "maxtemp_c": day.get("day", {}).get("maxtemp_c"),  # Max temperature
                "mintemp_c": day.get("day", {}).get("mintemp_c"),  # Min temperature
                "condition": day.get("day", {})
                .get("condition", {})
                .get("text"),  # Forecast condition
            }
            for day in forecast  # Loops through all forecast days
        ],
    }
    return flattened_data  # Returns the flattened data


# Main program
def fetch_weather_data():  # Main function to fetch and process weather data
    base_url = "http://api.weatherapi.com/v1/"  # Base URL for weather API
    location = (
        "Queretaro"  # You can replace with any city name based on your preference
    )
    weatherapikey = dbutils.secrets.get(
        scope="key-vault-scope", key="weather-api-key"
    )  # Retrieves API key from Databricks secrets

    # Get data from API
    current_weather = get_current_weather(
        base_url, weatherapikey, location
    )  # Gets current weather data
    forecast_weather = get_forecast_weather(
        base_url, weatherapikey, location, 3
    )  # Gets 3-day forecast data
    alerts = get_alerts(base_url, weatherapikey, location)  # Gets weather alerts

    # Flatten and merge data
    merged_data = flatten_data(
        current_weather, forecast_weather, alerts
    )  # Flattens and merges all data
    print(json.dumps(merged_data, indent=3))  # Prints merged data as formatted JSON
    send_event(merged_data)

# Calling the Main Program
fetch_weather_data()  # Executes the main function